<a href="https://colab.research.google.com/github/mobarakol/tutorial_notebooks/blob/main/tiny_imagenet(folder).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tiny-ImageNet
It contains 200 classes from full imagenet dataset.<br>
However, file structures are not for train and validation images. Train images are separated with a class-wise folder name where all validation images are a single folder. We need an additional propcessing to rearrange validation images into class-wise folders.

Donwload & unzip

In [1]:
! wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
! unzip -q tiny-imagenet-200.zip

--2022-04-29 17:14:17--  http://cs231n.stanford.edu/tiny-imagenet-200.zip
Resolving cs231n.stanford.edu (cs231n.stanford.edu)... 171.64.68.10
Connecting to cs231n.stanford.edu (cs231n.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248100043 (237M) [application/zip]
Saving to: ‘tiny-imagenet-200.zip’

tiny-imagenet-200.z 100%[===================>] 236.61M  51.0MB/s    in 5.7s    

2022-04-29 17:14:23 (41.8 MB/s) - ‘tiny-imagenet-200.zip’ saved [248100043/248100043]



Preprocessing to rearranged the validation set in folders

In [2]:
import pandas as pd

VALID_DIR = 'tiny-imagenet-200/val'
val_data = pd.read_csv(f'{VALID_DIR}/val_annotations.txt', sep='\t', 
                            header=None, names=['File', 'Class', 'X', 'Y', 'H', 'W'])

val_data.head()

,File,Class,X,Y,H,W
0,val_0.JPEG,n03444034,0,32,44,62
1,val_1.JPEG,n04067472,52,55,57,59
2,val_2.JPEG,n04070727,4,0,60,55
3,val_3.JPEG,n02808440,3,3,63,63
4,val_4.JPEG,n02808440,9,27,63,48


In [3]:
import os
VALID_DIR = 'tiny-imagenet-200/val'
val_img_dir = os.path.join(VALID_DIR, 'images')
fp = open(os.path.join(VALID_DIR, 'val_annotations.txt'), 'r')
data = fp.readlines()

# Mapping image file name with label name
val_img_dict = {}
for line in data:
    words = line.split('\t')
    val_img_dict[words[0]] = words[1]
fp.close()

# moving images into corresponding class folders
for img, folder in val_img_dict.items():
    newpath = (os.path.join(val_img_dir, folder))
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    if os.path.exists(os.path.join(val_img_dir, img)):
        os.rename(os.path.join(val_img_dir, img), os.path.join(newpath, img))


Training CNN model

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision
from torchvision import datasets
from torchvision import models
import torchvision.transforms as transforms
import os
import argparse
import copy
import random
import numpy as np
device = 'cuda' if torch.cuda.is_available() else 'cpu'
def seed_everything(seed=42):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
parser = argparse.ArgumentParser(description='CIFAR-10H Training')
parser.add_argument('--lr', default=0.1, type=float, help='learning rate')
parser.add_argument('--lr_schedule', default=0, type=int, help='lr scheduler')
parser.add_argument('--train_batch', default=512, type=int, help='batch size')
parser.add_argument('--valid_batch', default=1024, type=int, help='batch size')
parser.add_argument('--num_epoch', default=100, type=int, help='epoch number')
parser.add_argument('--num_classes', type=int, default=200, help='number classes')
args = parser.parse_args(args=[])

def train(model, trainloader, criterion, optimizer):
    model.train()
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

def test(model, testloader):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return correct / total

Training

In [ ]:
seed_everything()
VALID_DIR = 'tiny-imagenet-200/val'
TRAIN_DIR = 'tiny-imagenet-200/train'
transform_train = transforms.Compose([transforms.Resize(64), transforms.RandomHorizontalFlip(), 
        transforms.ToTensor(), transforms.Normalize((0.485, 0.456, 0.406,), (0.229, 0.224, 0.225,))]
        )
transform_test = transforms.Compose([transforms.Resize(64), transforms.ToTensor(),
        transforms.Normalize((0.485, 0.456, 0.406,), (0.229, 0.224, 0.225,))]
        )

train_dataset = datasets.ImageFolder(os.path.join(TRAIN_DIR),
            transform=transform_train,)
test_dataset = datasets.ImageFolder(os.path.join(val_img_dir),
    transform=transform_test,)


trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=args.train_batch, shuffle=True,
                                          num_workers=2, pin_memory=True)
testloader = torch.utils.data.DataLoader(test_dataset, batch_size=args.valid_batch, shuffle=False, 
                                         num_workers=2, pin_memory=True)
print('sample size- Train:%d, Validation:%d'%(len(train_dataset), len(test_dataset)))


model = models.resnet18(pretrained=True).to(device)
model.fc = torch.nn.Linear(model.fc.in_features, args.num_classes)
model.to(device)

optimizer = optim.SGD(model.parameters(), lr=args.lr, momentum=0.9, weight_decay=5e-4)
criterion = nn.CrossEntropyLoss()
best_epoch, best_acc = 0, 0
for epoch in range(args.num_epoch):
    for epoch in range(args.num_epoch):
        train(model, trainloader, criterion, optimizer)
        accuracy = test(model, testloader)
        if accuracy > best_acc:
            patience = 0
            best_acc = accuracy
            best_epoch = epoch
            best_model = copy.deepcopy(model)
            torch.save(best_model.state_dict(), 'best_model_tiny_imagenet.pth.tar')
        print('epoch: {}  acc: {:.4f}  best epoch: {}  best acc: {:.4f}'.format(
                epoch, accuracy, best_epoch, best_acc, optimizer.param_groups[0]['lr']))

sample size- Train:100000, Validation:10000


Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

epoch: 0  acc: 0.3041  best epoch: 0  best acc: 0.3041
epoch: 1  acc: 0.3886  best epoch: 1  best acc: 0.3886
epoch: 2  acc: 0.3916  best epoch: 2  best acc: 0.3916
epoch: 3  acc: 0.3715  best epoch: 2  best acc: 0.3916
